In [29]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
#df = pd.read_csv("noAudible/Features_10too200hz.csv")
df1 = pd.read_csv("noAudible/Features_5too10hz.csv")
df2 = pd.read_csv("noAudible/Features_10too15hz.csv")
df3 = pd.read_csv("noAudible/Features_15too20hz.csv")
df4 = pd.read_csv("noAudible/Features_20too25hz.csv")
df5 = pd.read_csv("noAudible/Features_25too30hz.csv")
df6 = pd.read_csv("noAudible/Features_30too35hz.csv")
#df6 = pd.read_csv("noAudible/Features_10hz_200hz_no_audible_mav2.csv")
frames =[df1,df2,df3,df4,df5,df6]
df = pd.concat(frames)



#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.004527,0.004230,0.005904,17.686214,0.136204,0.000035,1.441989,17.686214,0.000369,103,...,0.000477,102,105,0,53.984576,70.091919,1,1001,10010001,audible
1,0.004243,0.003844,0.006041,11.154593,0.095955,0.000036,0.883116,11.154593,0.000336,64,...,0.000540,64,68,0,62.068966,78.696236,1,1001,10010002,audible
2,0.004371,0.004153,0.006294,20.688032,0.187501,0.000040,1.591817,20.688032,0.000336,114,...,0.000501,116,126,0,59.745763,75.760144,1,1001,10010003,audible
3,0.004803,0.004578,0.006292,23.336935,0.192336,0.000040,1.853118,23.336935,0.000381,123,...,0.000502,120,124,0,61.363636,77.790422,1,1001,10010004,audible
4,0.004426,0.004217,0.005788,24.270778,0.183748,0.000034,1.932590,24.270778,0.000352,138,...,0.000429,132,142,0,63.071298,79.029418,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.003756,0.003385,0.005313,10.236235,0.076932,0.000028,3.462329,10.236235,0.001271,294,...,0.001137,297,296,0,18.819188,64.655443,8,8019,80190246,whisper
4796,0.002726,0.002237,0.003858,6.056701,0.033079,0.000015,2.068628,6.056701,0.000931,241,...,0.001041,239,246,0,31.221719,76.098426,8,8019,80190247,whisper
4797,0.002786,0.002370,0.004054,7.323362,0.043200,0.000016,2.504635,7.323362,0.000953,287,...,0.001025,284,285,0,26.436782,72.680663,8,8019,80190248,whisper
4798,0.002729,0.002472,0.003873,5.449347,0.029962,0.000015,1.851247,5.449347,0.000927,218,...,0.001087,216,218,0,24.242424,69.342802,8,8019,80190249,whisper


In [30]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [31]:
def lda():
    ids = [1001,1002,1003,2001,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032,4001,4002,7001,7002,8002,8003, 8010, 8016, 8017, 8018, 8019]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))

In [32]:
def lda_user1_session1():
    train_ids = [1002,1003]
    test_ids = [1001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score
    
def lda_user1_session2():
    train_ids = [1001,1003]
    test_ids = [1002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score
    
def lda_user1_session3():
    train_ids = [1001,1002]
    test_ids = [1003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score
    
def lda_user2_session1():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score  
    
def lda_user2_session3():
    train_ids = [2001, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score 
    
def lda_user2_session4():
    train_ids = [2003, 2001, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2004]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 4 is "+str(score))
    return score    
    
def lda_user2_session5():
    train_ids = [2003, 2004, 2001, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2005]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 5 is "+str(score))
    return score  
    
def lda_user2_session6():
    train_ids = [2003, 2004, 2005, 2001, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2006]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 6 is "+str(score))
    return score   
    
def lda_user2_session7():
    train_ids = [2003, 2004, 2005, 2006, 2001, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2007]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 7 is "+str(score))
    return score     
    
def lda_user2_session8():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2001, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2008]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 8 is "+str(score))
    return score 
    
def lda_user2_session9():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2001, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2009]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 9 is "+str(score))
    return score   
    
def lda_user2_session10():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2001, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 10 is "+str(score))
    return score     
    
def lda_user2_session12():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2001, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2012]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 12 is "+str(score))
    return score 
    
def lda_user2_session13():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2032] 
    test_ids = [2013]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 13 is "+str(score))
    return score   
    
def lda_user2_session28():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2013, 2029, 2031, 2032] 
    test_ids = [2028]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 28 is "+str(score))
    return score  
    
def lda_user2_session29():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2013, 2031, 2032] 
    test_ids = [2029]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 29 is "+str(score))
    return score     
    
def lda_user2_session31():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2013, 2032] 
    test_ids = [2031]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 31 is "+str(score))
    return score  
    
def lda_user2_session32():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2013] 
    test_ids = [2032]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 32 is "+str(score))
    return score 
    
def lda_user4_session1():
    train_ids = [4002]
    test_ids = [4001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score   
    
def lda_user4_session2():
    train_ids = [4001]
    test_ids = [4002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user7_session1():
    train_ids = [7002]
    test_ids = [7001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 1 is "+str(score))
    return score 
    
def lda_user7_session2():
    train_ids = [7001]
    test_ids = [7002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user8_session2():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 2 is "+str(score))
    return score   
    
def lda_user8_session3():
    train_ids = [8002, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 3 is "+str(score))
    return score  
    
def lda_user8_session10():
    train_ids = [8003, 8002, 8016, 8017, 8018, 8019]
    test_ids = [8010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 10 is "+str(score))
    return score   
    
def lda_user8_session16():
    train_ids = [8003, 8010, 8002, 8017, 8018, 8019]
    test_ids = [8016]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 16 is "+str(score))
    return score    
    
def lda_user8_session17():
    train_ids = [8003, 8010, 8016, 8002, 8018, 8019]
    test_ids = [8017]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 17 is "+str(score))
    return score  
    
def lda_user8_session18():
    train_ids = [8003, 8010, 8016, 8017, 8002, 8019]
    test_ids = [8018]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 18 is "+str(score))
    return score     
    
def lda_user8_session19():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8002]
    test_ids = [8019]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [33]:
def lda_all():
    print ("User 1 : " )
    a = lda_user1_session1()
    b = lda_user1_session2()
    c = lda_user1_session3()
    print ("User 2 : " )
    d = lda_user2_session1()
    e = lda_user2_session3()
    f = lda_user2_session4()
    g = lda_user2_session5()
    h = lda_user2_session6()
    i = lda_user2_session7()
    j = lda_user2_session8()
    k = lda_user2_session9()
    l = lda_user2_session10()
    m = lda_user2_session12()
    n = lda_user2_session13()
    o = lda_user2_session28()
    p = lda_user2_session29()
    q = lda_user2_session31()
    r = lda_user2_session32()
    print ("User 4 : " )
    s = lda_user4_session1()
    t = lda_user4_session2()
    print ("User 7 : " )
    u = lda_user7_session1()
    v = lda_user7_session2()
    print ("User 8 : " )
    w = lda_user8_session2()
    x = lda_user8_session3()
    y = lda_user8_session10()
    z = lda_user8_session16()
    aa = lda_user8_session17()
    bb = lda_user8_session18()
    cc = lda_user8_session19()
    scores = np.array([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc])
    print("Durchschnitt:")
    print((scores.mean()))
    print ("Standartabweichung")
    print(scores.std())


In [34]:
def ldas():
    ids = [1001,1002,1003,2001,2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,2011, 2012, 2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,
        2023,2024,2025,2026,2027,2028, 2029, 2030,2031, 2032,2101,4001,4002,7001,7002,8001,8002,8003,8004,8005,8006,8007,8008,8009, 8010, 8011,8012,8013,8014,8015, 8016, 8017, 8018,8019, 8101]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))
    print((scores.std()))

In [35]:
lda_all()

User 1 : 
Accuracy for session 1 is 0.7377777777777778
Accuracy for session 2 is 0.7822222222222223
Accuracy for session 3 is 0.6833333333333333
User 2 : 
Accuracy for session 1 is 0.37777777777777777
Accuracy for session 3 is 0.49
Accuracy for session 4 is 0.4388888888888889
Accuracy for session 5 is 0.3988888888888889
Accuracy for session 6 is 0.46444444444444444
Accuracy for session 7 is 0.4411111111111111
Accuracy for session 8 is 0.4588888888888889
Accuracy for session 9 is 0.41
Accuracy for session 10 is 0.47555555555555556
Accuracy for session 12 is 0.4811111111111111
Accuracy for session 13 is 0.24666666666666667
Accuracy for session 28 is 0.38222222222222224
Accuracy for session 29 is 0.3522222222222222
Accuracy for session 31 is 0.34555555555555556
Accuracy for session 32 is 0.32
User 4 : 
Accuracy for session 1 is 0.5266666666666666
Accuracy for session 2 is 0.62
User 7 : 
Accuracy for session 1 is 0.3433333333333333
Accuracy for session 2 is 0.36777777777777776
User 8 : 
Ac